In [1]:
# disable future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# safe imports
import os, sys       # arcgis comes with these
import datetime      # arcgis comes with this
import numpy as np   # arcgis comes with this
import pandas as pd  # arcgis comes with this

# risk imports (non-native to arcgis)
try:
    import xarray as xr  # not in arcgis
except:
    arcpy.AddError('Python library Xarray is not installed.')
    raise

# import tools
try:
    # shared folder
    sys.path.append(r'C:\Users\262272G\Documents\GitHub\tenement-tools\shared')
    sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\shared')
    import arc, satfetcher, tools  

    # module folder
    sys.path.append(r'C:\Users\262272G\Documents\GitHub\tenement-tools\modules')
    sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\modules')
    import vegfrax, cog  

except:
    arcpy.AddError('Could not find tenement tools python scripts (modules, shared).')
    raise

In [2]:
# we need a cube of data and we need to get a specific date or range of dates from it

In [3]:
# in_low_res_nc
# in_high_res_tif
# in_subset_years (all dates in this will be aggregated)
# in_aggregator (median, mean)
# in_set_classes
# in_merge_focus_classes
# in_num_samples

# in_num_estimators
# in_num_validations

# in_conform_nodata

# in_fmask_flags
# in_max_cloud
# in_add_result_to_map

# out_nc

In [50]:
ds = xr.open_rasterio(r"C:\Users\262272G\Desktop\test\classedliot_tif.tif")
#ds = ds.to_dataset(name='variable')

In [ ]:
ds.where(ds['time'] >= dts & )

In [68]:
from datetime import datetime

start_dt = np.datetime64(datetime.utcnow()).astype(datetime)

In [88]:
ds = xr.open_dataset(r"C:\Users\262272G\Desktop\test\out_ls.nc")

In [87]:
dts = ds['time'].values
dts > np.datetime64('2018-01-05T02:03:11.000000000')

array([False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [ ]:
        # convert datetime strings to numpy datetime64
        in_from_date = np.datetime64(in_from_date).astype(datetime.datetime)
        in_to_date = np.datetime64(in_to_date).astype(datetime.datetime)